# Import learning data

In [1]:
import pandas as pd

training_data1_path = "../../learning_data/data/win_rate/flop/50000-data-1000-simulation-2-players-win-rate-data.csv"
training_data2_path = "../../learning_data/data/win_rate/flop/100000-data-1000-simulation-2-players-win-rate-data.csv"
test_data_path = "../../learning_data/data/win_rate/flop/10000-data-1000-simulation-2-players-win-rate-data.csv"
train1_df = pd.read_csv(training_data1_path)
train2_df = pd.read_csv(training_data2_path)
train_df = train1_df.append(train2_df, ignore_index=True)
test_df = pd.read_csv(test_data_path)

# About learning data

In [2]:
print train_df.shape, test_df.shape

(150000, 11) (10000, 11)


In [3]:
train_df.head()

,hole1,hole2,community1,community2,community3,win_rate,hole1_id,hole2_id,community1_id,community2_id,community3_id
0,D8,CA,DQ,S3,CT,0.530,21,1,25,42,10
1,S5,HJ,C3,D9,HQ,0.464,44,37,3,22,38
2,SA,S6,CK,H3,S5,0.564,40,45,13,29,44
3,HT,H5,C9,H9,C4,0.659,36,31,9,35,4
4,S8,HJ,CQ,HQ,DK,0.598,47,37,12,38,26


In [4]:
train_df.describe()

,win_rate,hole1_id,hole2_id,community1_id,community2_id,community3_id
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,0.585045,26.498753,26.606080,26.486640,26.467327,26.529093
std,0.154497,15.011237,15.013962,15.008892,14.990036,14.990617
min,0.248000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.461000,13.000000,14.000000,13.000000,13.000000,14.000000
50%,0.578000,26.000000,27.000000,26.000000,26.000000,26.000000
75%,0.679000,39.000000,40.000000,40.000000,39.000000,40.000000
max,1.000000,52.000000,52.000000,52.000000,52.000000,52.000000


In [5]:
test_df.describe()

,win_rate,hole1_id,hole2_id,community1_id,community2_id,community3_id
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.586272,26.592700,26.486300,26.434500,26.454800,26.452700
std,0.154742,15.059531,15.019759,15.042777,14.962638,15.058481
min,0.273000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.461000,13.000000,13.000000,13.000000,13.000000,13.000000
50%,0.579000,27.000000,27.000000,27.000000,26.000000,26.000000
75%,0.681000,40.000000,40.000000,39.000000,40.000000,40.000000
max,1.000000,52.000000,52.000000,52.000000,52.000000,52.000000


# Data Processing

## card id -> 1-hot formated vector

In [6]:
import numpy as np
from pypokerengine.engine.card import Card

fetch_hole = lambda row: [row[key] for key in ['hole1_id', 'hole2_id']]
fetch_community = lambda row: [row[key] for key in ['community1_id', 'community2_id', 'community3_id']]
gen_card_rank_vec = lambda card: [1 if r == card.rank else 0 for r in range(2,15)]
gen_card_suit_vec = lambda card: [1 if s == card.suit else 0 for s in [Card.CLUB, Card.DIAMOND, Card.HEART, Card.SPADE]]
gen_card_vec = lambda card: gen_card_rank_vec(card) + gen_card_suit_vec(card)
gen_img = lambda zipped: [gen_card_vec(Card.from_id(card_id)) for card_id in zipped[0]+zipped[1]]

train_hole = train_df.apply(lambda row: fetch_hole(row), axis=1)
train_community = train_df.apply(lambda row: fetch_community(row), axis=1)
train_df["onehot"] = map(gen_img, zip(train_hole, train_community))

test_hole = test_df.apply(lambda row: fetch_hole(row), axis=1)
test_community = test_df.apply(lambda row: fetch_community(row), axis=1)
test_df["onehot"] = map(gen_img, zip(test_hole, test_community))

## Format data (pandas.df -> numpy.ndarray)

In [7]:
wrap = lambda lst: np.array(lst)
to_ndarray = lambda X: wrap([wrap(x) for x in X])
train_x, train_y = [to_ndarray(array) for array in [train_df["onehot"].values, train_df["win_rate"].values]]
test_x, test_y = [to_ndarray(array) for array in [test_df["onehot"].values, test_df["win_rate"].values]]
train_x, test_x = [X.reshape(X.shape[0], 1, X.shape[1], X.shape[2]) for X in [train_x, test_x]]
print "shape of training data => x: %s, y: %s" % (train_x.shape, train_y.shape)
print "shape of test data => x: %s, y: %s" % (test_x.shape, test_y.shape)

shape of training data => x: (150000, 1, 5, 17), y: (150000,)
shape of test data => x: (10000, 1, 5, 17), y: (10000,)


# Create model

In [8]:
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core import Dense, Activation, Dropout, Flatten

model = Sequential()

model.add(Convolution2D(128, 3, 3, border_mode="same", input_shape=(1, 5, 17)))
model.add(Activation("relu"))
model.add(Convolution2D(128, 3, 3, border_mode="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(128, 3, 3, border_mode="same", input_shape=(1, 5, 17)))
model.add(Activation("relu"))
model.add(Convolution2D(128, 3, 3, border_mode="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1))
model.compile(loss="mse",  optimizer="adadelta")

Using Theano backend.


# Train or Load existing model

In [9]:
import os
model_weight_file_path = "./resource/flop_cnn_model_weights.h5"
if os.path.exists(model_weight_file_path):
    model.load_weights(model_weight_file_path)
else:
    history = model.fit(train_x, train_y, batch_size=128, nb_epoch=1, validation_split=0.1, verbose=2)

# Check model performance

## Visualize loss transition

In [10]:
%matplotlib inline
import seaborn
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for idx, key in enumerate(history.history, start=1):
    plt.subplot(2, 1, idx)
    plt.plot(range(len(history.history[key])), history.history[key])
    plt.title(key)
plt.show()

<img src="./resource/flop_cnn_loss_history.png" width=800/>

## Test model performance by MSE 

In [11]:
from sklearn.metrics import mean_squared_error

def print_model_performance(model, train_x, train_y, test_x, test_y):
    print 'MSE on training data = {score}'.format(score=mean_squared_error(model.predict(train_x), train_y))
    print 'MSE on test data = {score}'.format(score=mean_squared_error(model.predict(test_x), test_y))

In [12]:
print_model_performance(model, train_x, train_y, test_x, test_y)

MSE on training data = 0.00017142599317
MSE on test data = 0.000307717757026


## See model prediction on sample data

In [13]:
from pypokerengine.engine.card import Card
C, D, H, S = Card.CLUB, Card.DIAMOND, Card.HEART, Card.SPADE

test_case = [
    [(4, C), (10, S), (10, D), (1, C), (10, H), 0.954],
    [(4, S), (2, S), (12, D), (11, C), (10, D), 0.274],
    [(1, H), (11, H), (8, D), (3, C), (7, D), 0.507]
]

to_id = lambda card: card.to_id()
gen_card_rank_vec = lambda card: [1 if r == card.rank else 0 for r in range(2,15)]
gen_card_suit_vec = lambda card: [1 if s == card.suit else 0 for s in [Card.CLUB, Card.DIAMOND, Card.HEART, Card.SPADE]]
gen_card_vec = lambda card: gen_card_rank_vec(card) + gen_card_suit_vec(card)
gen_img = lambda zipped: [gen_card_vec(Card.from_id(card_id)) for card_id in zipped[0]+zipped[1]]
wrap = lambda lst: np.array(lst)
to_ndarray = lambda X: wrap([wrap(x) for x in X])

for card1, card2, card3, card4, card5, expected in test_case:
    cards = [Card(rank=rank, suit=suit) for rank, suit in [card1, card2, card3, card4, card5]]
    hole = cards[:2]
    community = cards[2:]
    hole_ids = map(to_id, hole)
    community_ids = map(to_id, community)
    x = gen_img((hole_ids, community_ids))
    X = to_ndarray(x)
    X = np.array([X.reshape(1, X.shape[0], X.shape[1])])
    y = model.predict(X)[0][0]
    print "HOLE = [%s, %s], COMMUNITY = [%s, %s, %s] => win_rate = { prediction=%f, expected=%f }" % tuple(map(str, hole) + map(str, community) + [y , expected])

HOLE = [C4, ST], COMMUNITY = [DT, CA, HT] => win_rate = { prediction=0.962527, expected=0.954000 }
HOLE = [S4, S2], COMMUNITY = [DQ, CJ, DT] => win_rate = { prediction=0.296932, expected=0.274000 }
HOLE = [HA, HJ], COMMUNITY = [D8, C3, D7] => win_rate = { prediction=0.502538, expected=0.507000 }
